In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_raw_data, get_tags_as_inputs
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

                                                    tags  category_id
0      Eminem.Walk.On.Water.Aftermath/Shady/Interscop...           10
1      plush.bad unboxing.unboxing.fan mail.idubbbztv...           23
2      racist superman.rudy.mancuso.king.bach.racist....           23
3      ryan.higa.higatv.nigahiga.i dare you.idy.rhpc....           24
4      edsheeran.ed sheeran.acoustic.live.cover.offic...           10
...                                                  ...          ...
40876  annoying orange.funny.fruit.talking.animation....           24
40877  hkayet tounsia.elhiwar ettounsi.denya okhra.ou...           24
40878  Kingdom Hearts.KH3.Kingdom Hearts 3.Frozen.Squ...           20
40879  180612__TB02SorryExcuse.News.Politics.The Youn...           25
40880  吳宗憲.吳姍儒.小明星大跟班.Sandy.Jacky wu.憲哥.中天綜合台.CH36.中天...           24

[40881 rows x 2 columns]


In [79]:
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# tab_dataframe.dropna(inplace = True) 
# tab_dataframe["tags"]= tab_dataframe["tags"].str.split('.')

# Creating different coloumn based on different tags

new = tab_dataframe["tags"].str.split(".", expand = True)

for i in range(0, 124):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old Name columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("NA", inplace = True) 


tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags114,tags115,tags116,tags117,tags118,tags119,tags120,tags121,tags122,tags123
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [80]:
vocab = get_vocab(get_tags(csvpath))

# print first 10 words from vocab
for word in vocab[:11]:
    print(word)


dollar store
making projectiles
eepfake
langue collé sur un poteau
weezy
oculus
Are Energy Drinks Bad For You
Roti
fedexfield
sunny day flooding


In [81]:
train, test = train_test_split(tab_dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


26163 train examples
6541 validation examples
8177 test examples


In [82]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('category_id')
    (dataframe)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


In [83]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

